<a href="https://colab.research.google.com/github/jinsusong/study-Kaggle/blob/main/Classification_Spooky_Author_Identification_Kaggle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Kaggle Spooky Author Identification 데이터 불러오기 

In [ ]:
!pip install kaggle
from google.colab import files
files.upload()

In [ ]:
#!pip install --upgrade --force-reinstall --no-deps kaggle

In [ ]:

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

# Permission Warning 방지
!chmod 600 ~/.kaggle/kaggle.json
!pwd

In [ ]:
!kaggle competitions download -c spooky-author-identification

In [ ]:
!ls -al

# 데이터 압축 해제

In [ ]:
!mkdir data
!unzip sample_submission.zip -d data/
!unzip test.zip -d data/
!unzip train.zip -d data/

# 데이터 확인하기 

In [ ]:
import pandas as pd 

label_cols = ['author']
train = pd.read_csv("data/train.csv")
print(len(train))
#len_eap = train['author'] == 'EAP'
#len_eap
train.head()



In [ ]:

test = pd.read_csv("data/test.csv")
test.head()


#Preprocess Data 

In [ ]:
from sklearn.model_selection import train_test_split
X = list(train["text"])
y = list(train["author"])
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1)

#X_train, X_test, y_train, y_test = train_test_split(bert_train['text'], bert_train['target'], test_size=0.25, random_state=42)

In [ ]:
#X_train
#y_train

In [ ]:
# train['author']
# labels =  train['author'].values
# print(labels)

# train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids, labels, random_state=0, test_size=0.1)
# train_masks, validation_masks, _, _ = train_test_split(attention_masks, labels, random_state=0, test_size=0.1)

# train_size = len(train_inputs)
# validation_size = len(validation_inputs)


In [ ]:
!pip install transformers

# 버트 토크나이저 로드 

In [ ]:
from transformers import AutoTokenizer

bert_model_name = 'bert-base-uncased'

tokenizer = AutoTokenizer.from_pretrained(bert_model_name)


#Tokenized

In [ ]:
train_encodings = tokenizer(X_train, padding=True, truncation=True, max_length=512,add_special_tokens=True, return_attention_mask=True) 
val_encodings = tokenizer(X_val, padding=True, truncation=True, max_length=512)
test_encodings = tokenizer(y_train, padding=True, truncation=True, max_length=512,add_special_tokens=True, return_attention_mask=True) 
test_val_encodings = tokenizer(y_val, padding=True, truncation=True, max_length=512)
len(train_encodings)


# input_ids, token_type_ids, attention_mask = (input_dict["input_ids"],input_dict["token_type_ids"], input_dict['attention_mask'])

In [ ]:
train_encodings = tokenizer(X_train, padding=True, truncation=True, max_length=512,add_special_tokens=True, return_attention_mask=True) 

In [ ]:
x_test_input_ids, x_test_token_type_ids, x_test_attention_mask = (train_encodings["input_ids"],train_encodings["token_type_ids"], train_encodings['attention_mask'])
x_val_input_ids, x_val_token_type_ids, x_val_attention_mask = (val_encodings["input_ids"],val_encodings["token_type_ids"], val_encodings['attention_mask'])
y_test_input_ids, y_test_token_type_ids, y_test_attention_mask = (test_encodings["input_ids"],test_encodings["token_type_ids"], test_encodings['attention_mask'])
y_val_input_ids, y_val_token_type_ids, y_val_attention_mask = (test_val_encodings["input_ids"],test_val_encodings["token_type_ids"], test_val_encodings['attention_mask'])


In [ ]:
print(x_test_input_ids[0])
print(x_test_token_type_ids[0])
print(x_test_attention_mask[0])
print(y_test_input_ids[0])

#데이터 셋으로 변환

In [ ]:
# import tensorflow as tf
# train_data = tf.data.Dataset.from_tensor_slices(train_encodings)
# train_val_data = tf.data.Dataset.from_tensor_slices(val_encodings)
# y_data = tf.data.Dataset.from_tensor_slices(test_encodings)
# y_val_data = tf.data.Dataset.from_tensor_slices(test_val_encodings)


import tensorflow as tf
train_data = tf.data.Dataset.from_tensor_slices((x_test_input_ids, x_test_token_type_ids, x_test_attention_mask))
train_val_data = tf.data.Dataset.from_tensor_slices((x_val_input_ids, x_val_token_type_ids, x_val_attention_mask))
y_data = tf.data.Dataset.from_tensor_slices((y_test_input_ids, y_test_token_type_ids, y_test_attention_mask))
y_val_data = tf.data.Dataset.from_tensor_slices((y_val_input_ids, y_val_token_type_ids, y_val_attention_mask))



In [ ]:
# 데이터 셋 batch 사용해보기
print(train_data)
batch_size = 32
batch_train_data = train_data.batch(batch_size)
batch_train_val_data = train_val_data.batch(batch_size)
batch_y_data = y_data.batch(batch_size)
batch_y_val_data = y_val_data.batch(batch_size)

print(batch_train_data)

In [ ]:
# y label 데이터 셋에 포함 시키기 , 실패 , 학습이 안됨
# 학습 시  y 데이터를 넣어야함 ㅜㅜ
# 데이터 형태 문제인듯, y label을 어떻게 넣는지 모르겠다.
train_data = tf.data.Dataset.from_tensor_slices((train_encodings["input_ids"], train_encodings["token_type_ids"],train_encodings['attention_mask'], test_encodings["input_ids"]))
# train_val_data = tf.data.Dataset.from_tensor_slices((x_val_input_ids, x_val_token_type_ids, y_val_input_ids))

In [ ]:
train_data

In [ ]:
len(train_data)
i = 0
for element in train_data:
  i += 1
  print(element)
  if i > 1 : break

# Training the Model 

In [ ]:
from transformers import TFBertForSequenceClassification

model = TFBertForSequenceClassification.from_pretrained(bert_model_name)

In [ ]:
# train_data 
# train_val_data 
# y_data 
# y_val_data 

# loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
# model.compile(optimizer='adam',loss=loss)

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0), 
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy('accuracy')])


In [ ]:
model.summary

In [ ]:
# 모델의 구조 확인하기 
tf.keras.utils.plot_model(model)

In [ ]:
model.fit(train_data, epochs=1, batch_size=32)
#model.fit(batch_train_data, validation_data=train_val_data, epochs=1)